In [6257]:
import pandas as pd

In [6258]:
trade_train = pd.read_csv("./trade_train.csv")

In [6259]:
stock = pd.read_csv("./stocks.csv")

In [6260]:
trade_train.drop("Unnamed: 0",inplace=True,axis=1)

In [6261]:
stock.drop("index",axis=1,inplace=True)

In [6262]:
stock['industry code'] = stock['표준산업구분코드_대분류'].apply(lambda x:str(x)) + stock['표준산업구분코드_중분류'].apply(lambda x:str(x)) + stock['표준산업구분코드_소분류'].apply(lambda x:str(x))

In [6263]:
def makehangul(x):
    result = ""
    for i in x:
        if i>='가' and i<='힣':
            result+=i
    return result

In [6264]:
stock['industry code']=stock['industry code'].apply(makehangul)

In [6265]:
code_table = pd.read_excel("./한국표준산업분류(10차)_표 (1).xlsx")

In [6266]:
code_table = code_table.ffill()

In [6267]:
code_table['industry code'] = code_table[1].apply(lambda x:str(x)) + code_table[2].apply(lambda x:str(x)) + code_table[3].apply(lambda x:str(x))

In [6268]:
code_table['industry code'] = code_table['industry code'].apply(makehangul)

In [6269]:
code_table = code_table[['industry code','코드']]

In [6270]:
new_stock = pd.merge(stock,code_table,how='left')

In [6271]:
len(new_stock[pd.isna(new_stock['코드'])]) / len(new_stock) * 100

23.598113689038353

In [6272]:
code_na_list = list(new_stock[pd.isna(new_stock['코드'])]['industry code'].unique())

In [6273]:
code_na_list

['제조업전자부품컴퓨터영상음향및반도체제조업',
 '제조업의약품제외기타화학제품제조업',
 '제조업화학물질및화학제품제조업기초화학물질제조업',
 '제조업의약품제외비료농약및살균살충제제조업',
 '제조업통신장비제조업전자부품제조업',
 '제조업의복의복액세서리및모피제품봉제의복제조업',
 '제조업통신장비제조업통신및방송장비제조업',
 '제조업금속가공제품제조업기계및구조용금속제품탱크및증기발생기제조업',
 '제조업가구제외기타금속가공제품제조업',
 '제조업통신장비제조업영상및음향기기제조업',
 '제조업가구제외무기및총포탄제조업',
 '제조업통신장비제조업컴퓨터및주변장치제조업',
 '개인서비스업기타개인서비스업그외기타개인서비스업',
 '전문과학및기술서비스업건축기술엔지니어링및기타건축기술엔지니어링및관련기술서비스업',
 '예술스포츠및여가관련창작예술및여가관련서비스업창작및예술관련서비스업',
 '서비스업스포츠및오락관련서비스업유원지및기타오락관련서비스업',
 '제조업의약품제외합성고무및플라스틱물질제조업',
 '사업시설관리및사업지원및임대서비스업사업시설관리및조경서비스업사업시설유지관리서비스업',
 '사업시설관리및사업지원및임대서비스업사업지원서비스업여행사및기타여행보조서비스업',
 '전문과학및기술서비스업과학기술서비스업기타과학기술서비스업',
 '제조업의약품제외화학섬유제조업',
 '서비스업스포츠및오락관련서비스업스포츠서비스업']

In [6274]:
code_na_list_val = [
    260,
    200,
    201,
    203,
    260,
    141,
    264,
    251,
    259,
    265,
    252,
    263,
    969,
    721,
    901,
    912,
    202,
    741,
    752,
    730,
    205,
    911
]

In [6275]:
code_na_list_df = pd.DataFrame([code_na_list,code_na_list_val]).T

In [6276]:
code_na_list_df.columns=['industry code','코드']

In [6277]:
new_stock = pd.merge(new_stock,code_na_list_df,how='left',on='industry code')

In [6278]:
new_stock['산업코드'] = new_stock['코드_x'].fillna(0) + new_stock['코드_y'].fillna(0)

In [6279]:
new_stock.drop(['코드_x','코드_y','표준산업구분코드_대분류','표준산업구분코드_중분류','표준산업구분코드_소분류','industry code'],axis=1,inplace=True)

In [6280]:
new_stock['시장구분'] = new_stock['시장구분'].apply(lambda x:1 if x == '코스피' else 0)

In [6281]:
flow = (new_stock.pivot_table('종목시가',index=['기준일자'],columns=['종목번호'])+1).pct_change().clip(-0.35,0.35).fillna(0)

In [6282]:
trade_train['매수비율'] = trade_train['매수고객수'] / trade_train['그룹내고객수']

In [6283]:
new_stock['기준년월'] = new_stock['기준일자'].apply(lambda x:str(x)[:6])

In [6284]:
for i in new_stock.columns[4:12]:
    new_stock[i] = pd.to_numeric(new_stock[i])

In [6285]:
new_stock.drop("기준일자",axis=1,inplace=True)

In [6286]:
stock_mean = new_stock.groupby(["기준년월",'종목번호']).mean()

In [6287]:
stock_std = new_stock.groupby(["기준년월",'종목번호']).std()

In [6288]:
stock_cv = stock_std / stock_mean

In [6289]:
stock_mean.columns = [str(x) + "_mean" for x in list(stock_mean.columns)]

In [6290]:
stock_std.drop(["시장구분","산업코드"],axis=1,inplace=True)

In [6291]:
stock_std.columns = [str(x) + "_std" for x in list(stock_std.columns)]

In [6292]:
stock_cv.drop(['시장구분','산업코드'],axis=1,inplace=True)

In [6293]:
stock_cv.columns = [str(x) + "_cv" for x in list(stock_cv.columns)]

In [6294]:
stock_monthly_rtn = new_stock.groupby(["기준년월",'종목번호']).last().iloc[:,2:] / new_stock.groupby(["기준년월",'종목번호']).first().iloc[:,2:]

In [6295]:
stock_monthly_rtn.drop(["시장구분","산업코드"],axis=1,inplace=True)

In [6296]:
stock_monthly_rtn.columns = [str(x)+"_monthly_rtn" for x in list(stock_monthly_rtn.columns)]

In [6297]:
train = pd.concat([stock_mean,stock_std,stock_cv,stock_monthly_rtn],axis=1,join='outer')

In [6298]:
new_flow = flow.copy()

In [6299]:
new_flow.index = [str(x)[:6] for x in new_flow.index]

In [6300]:
stock_monthly_max_rtn = pd.DataFrame(new_flow.groupby(new_flow.index).max().T.unstack())

In [6301]:
stock_monthly_max_rtn.columns=['rtn_max']

In [6302]:
stock_monthly_mean_rtn = pd.DataFrame(new_flow.groupby(new_flow.index).mean().T.unstack())

In [6303]:
stock_monthly_mean_rtn.columns=['rtn_mean']

In [6304]:
stock_monthly_mean_std = pd.DataFrame(new_flow.groupby(new_flow.index).std().T.unstack())

In [6305]:
stock_monthly_mean_std.columns=['rtn_std']

In [6306]:
train

시장구분_mean  ...  거래금액_만원단위_monthly_rtn
기준년월   종목번호                ...                       
201907 A000020        1.0  ...               1.552075
       A000080        1.0  ...               3.050365
       A000100        1.0  ...               0.106820
       A000120        1.0  ...               0.665179
       A000140        1.0  ...             376.845853
...                   ...  ...                    ...
202007 A336370        1.0  ...               8.308645
       A33637K        1.0  ...               5.199993
       A33637L        1.0  ...               4.042150
       A344820        1.0  ...               0.511197
       A353810        0.0  ...               0.519928

[13912 rows x 26 columns]

train = pd.merge(train.reset_index(),stock_monthly_rtn.reset_index(),how='left')

train = pd.merge(train,stock_monthly_max_rtn.reset_index(),how='left')

train = pd.merge(train,stock_monthly_mean_std.reset_index(),how='left')

In [6307]:
train = pd.concat([train,stock_monthly_mean_rtn,stock_monthly_max_rtn,stock_monthly_mean_std],axis=1,join='outer')

In [6308]:
train = train.reset_index()

In [6309]:
train.columns = ['기준년월','종목번호'] + list(train.columns)[2:]

In [6310]:
test = trade_train[trade_train['기준년월']==202006]

In [6311]:
test['매수비율'] = 0

<ipython-input-6311-7df9416b17c9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['매수비율'] = 0


In [6312]:
test['기준년월'] = 202007

<ipython-input-6312-de8023d31cf4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['기준년월'] = 202007


In [6313]:
trade_train = pd.concat([trade_train,test])

In [6314]:
trade_train = trade_train[['기준년월','그룹번호','종목번호','그룹내고객수','매수비율']]

In [6315]:
trade_train['기준년월'] = trade_train['기준년월'].apply(lambda x:str(x))

In [6316]:
_train = pd.merge(trade_train,train,on=['기준년월','종목번호'],how='left')

In [6317]:
def shift_month(x):
    if x == "201907":
        return "201908"
    if x == "201908":
        return "201909"
    if x == "201909":
        return "201910"
    if x == "201910":
        return "201911"
    if x == "201911":
        return "201912"
    if x == "201912":
        return "202001"
    if x == "202001":
        return "202002"
    if x == "202002":
        return "202003"
    if x == "202003":
        return "202004"
    if x == "202004":
        return "202005"
    if x == "202005":
        return "202006"
    if x == "202006":
        return "202007"

In [6318]:
shifted = trade_train.copy()
shifted['기준년월'] = shifted['기준년월'].apply(shift_month)

In [6319]:
shifted = shifted.dropna(axis=0)

In [6320]:
shifted.drop("그룹내고객수",axis=1,inplace=True)

In [6321]:
_train = pd.merge(_train,shifted,how='left',on=['기준년월','그룹번호','종목번호']).fillna(0)

In [6322]:
search_data = pd.read_csv("검색데이터.csv")

In [6323]:
search_data.set_index("Unnamed: 0",inplace=True,drop=True)

In [6324]:
stock_name_list = stock[['종목명','종목번호']].drop_duplicates()

In [6325]:
stock_name_list.set_index("종목명",drop=True,inplace=True)

In [6326]:
stock_name_list

,종목번호
종목명,
동화약품,A000020
하이트진로,A000080
유한양행,A000100
CJ대한통운,A000120
하이트진로홀딩스,A000140
...,...
플레이디,A237820
엔피디,A198080
이지바이오,A353810


In [6327]:
search_data.columns = [stock_name_list.loc[x,'종목번호'] for x in search_data.columns]

In [6328]:
search_data = search_data.iloc[:-1,:]

In [6329]:
search_data.index = [ str(i).replace("-","")[:6] for i in search_data.index ]

In [6330]:
search_data_pct_change = search_data.pct_change(1).fillna(0)

In [6331]:
search_data_pct_change = search_data_pct_change.unstack().reset_index()

In [6332]:
search_data_pct_change.columns=['종목번호','기준년월','검색변화율']

In [6333]:
_train

,기준년월,그룹번호,종목번호,그룹내고객수,매수비율_x,시장구분_mean,종목시가_mean,종목고가_mean,종목저가_mean,종목종가_mean,거래량_mean,거래금액_만원단위_mean,산업코드_mean,종목시가_std,종목고가_std,종목저가_std,종목종가_std,거래량_std,거래금액_만원단위_std,종목시가_cv,종목고가_cv,종목저가_cv,종목종가_cv,거래량_cv,거래금액_만원단위_cv,종목시가_monthly_rtn,종목고가_monthly_rtn,종목저가_monthly_rtn,종목종가_monthly_rtn,거래량_monthly_rtn,거래금액_만원단위_monthly_rtn,rtn_mean,rtn_max,rtn_std,매수비율_y
0,201907,MAD01,A000660,288,0.024306,1.0,74391.304348,75608.695652,73443.478261,74539.130435,2.993388e+06,2.244919e+07,260.0,3967.013193,3934.799642,3765.733791,3852.005278,9.263853e+05,7.441696e+06,0.053326,0.052042,0.051274,0.051678,0.309477,0.331491,1.098592,1.114085,1.080344,1.098571,1.049194,1.148590,0.004346,0.045321,0.022784,0.000000
1,201907,MAD01,A001820,288,0.010417,1.0,49143.478261,50328.260870,47967.391304,48756.521739,3.604345e+05,1.821396e+06,260.0,3547.576136,3740.315558,3533.495845,3676.161052,4.697868e+05,2.492288e+06,0.072188,0.074318,0.073665,0.075398,1.303390,1.368340,0.861328,0.842750,0.818359,0.843870,1.987147,1.643043,-0.005577,0.131715,0.043531,0.000000
2,201907,MAD01,A004020,288,0.010417,1.0,40245.652174,40650.000000,39836.956522,40197.826087,2.563905e+05,1.031642e+06,241.0,1067.485687,1017.349497,1015.696961,1045.279243,6.421439e+04,2.674552e+05,0.026524,0.025027,0.025496,0.026003,0.250455,0.259252,0.918824,0.909302,0.912426,0.907602,0.746717,0.678284,-0.003587,0.019083,0.013425,0.000000
3,201907,MAD01,A005380,288,0.013889,1.0,135304.347826,136934.782609,133217.391304,134826.086957,5.077075e+05,6.836587e+06,301.0,4179.873667,4296.680399,4002.346150,4241.592474,2.335818e+05,3.121968e+06,0.030892,0.031378,0.030044,0.031460,0.460072,0.456656,0.893993,0.905263,0.906810,0.897163,1.486736,1.347463,-0.004754,0.025830,0.014860,0.000000
4,201907,MAD01,A005930,288,0.045139,1.0,46276.086957,46619.565217,45865.217391,46215.217391,7.990078e+06,3.696300e+07,264.0,710.612662,642.038503,676.628470,735.686875,2.106385e+06,9.774596e+06,0.015356,0.013772,0.014753,0.015919,0.263625,0.264443,0.975713,0.983122,0.972973,0.973176,1.130838,1.105007,-0.001004,0.017563,0.011626,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35110,202007,MAD48,A207940,163,0.000000,1.0,755600.000000,764700.000000,741800.000000,751750.000000,1.999277e+05,1.507454e+07,211.0,18397.368233,18876.607635,16385.487803,17534.552356,6.403423e+04,4.977229e+06,0.024348,0.024685,0.022089,0.023325,0.320287,0.330174,0.985806,0.983418,1.010638,1.011968,0.744472,0.744076,-0.001016,0.048877,0.024134,0.018405
35111,202007,MAD48,A272210,163,0.000000,1.0,9112.500000,9213.500000,8996.500000,9106.000000,4.495869e+05,4.156590e+05,260.0,246.403072,282.102538,208.409390,265.119635,3.187382e+05,3.113237e+05,0.027040,0.030618,0.023166,0.029115,0.708958,0.748988,1.105499,1.101770,1.096481,1.100679,1.974541,2.167452,0.004848,0.042324,0.014804,0.018405
35112,202007,MAD48,A285130,163,0.000000,1.0,174100.000000,188175.000000,166400.000000,177500.000000,1.800092e+06,3.880514e+07,202.0,60315.966290,70723.381639,55265.769362,59359.076812,1.754405e+06,5.243438e+07,0.346444,0.375838,0.332126,0.334417,0.974620,1.351223,2.316602,2.290323,2.282258,2.158672,0.404047,0.895064,0.044161,0.350000,0.119078,0.036810
35113,202007,MAD48,A316140,163,0.000000,1.0,8830.500000,8921.500000,8723.000000,8808.500000,1.533328e+06,1.347315e+06,649.0,182.077919,169.807166,161.964908,162.716253,5.619225e+05,4.802140e+05,0.020619,0.019033,0.018568,0.018473,0.366472,0.356423,0.990940,0.971271,0.977350,0.972004,2.490301,2.426683,0.000395,0.018326,0.015063,0.030675


In [6334]:
_train = pd.merge(_train,search_data_pct_change,how='left')

In [6335]:
ant_file_list = sorted(["개인매수201907.csv","개인매수201911.csv","개인매수202003.csv","개인매수202007.csv","개인매수201908.csv","개인매수201912.csv","개인매수202004.csv","개인매수201909.csv","개인매수202001.csv","개인매수202005.csv","개인매수201910.csv","개인매수202002.csv","개인매수202006.csv"])

In [6336]:
ant_file_list

['개인매수201907.csv',
 '개인매수201908.csv',
 '개인매수201909.csv',
 '개인매수201910.csv',
 '개인매수201911.csv',
 '개인매수201912.csv',
 '개인매수202001.csv',
 '개인매수202002.csv',
 '개인매수202003.csv',
 '개인매수202004.csv',
 '개인매수202005.csv',
 '개인매수202006.csv',
 '개인매수202007.csv']

In [6337]:
ant_buy_list = []
for i in ant_file_list:
    tmp = pd.read_csv("./개인매수/"+i,encoding='cp949',usecols=[1,3])
    tmp['기준년월'] = int(i[4:10])
    ant_buy_list.append(tmp)

In [6338]:
ant_buy = pd.concat(ant_buy_list)

In [6339]:
ant_buy['순매수금액(백만)'] = ant_buy['순매수금액(백만)'].apply(lambda x:int(x.replace("+","").replace(",","")))

In [6340]:
ant_buy = pd.merge(ant_buy,stock_name_list.reset_index(),how='left')

In [6341]:
ant_buy['기준년월'] = ant_buy['기준년월'].apply(lambda x:str(x))

In [6342]:
ant_buy.drop("종목명",axis=1,inplace=True)

In [6343]:
ant_buy.columns = ['순매수금액(만)', '기준년월', '종목번호']
ant_buy['순매수금액(만)'] = ant_buy['순매수금액(만)'] / 100

In [6344]:
_train = pd.merge(_train,ant_buy,how='left',on=['종목번호','기준년월'])

In [6345]:
_train = _train.fillna(0)

In [6346]:
from lightgbm import LGBMRegressor

In [6347]:
stock_name_list = pd.DataFrame(_train['종목번호'].unique()).reset_index().set_index(0)

In [6348]:
stock_name_list_reverse = pd.DataFrame(_train['종목번호'].unique())

In [6349]:
stock_name_list.head(1)

,index
0,
A000660,0


In [6350]:
_train['종목번호'] = _train['종목번호'].apply(lambda x:stock_name_list.loc[x,"index"])

In [6351]:
_train['그룹번호'] = _train['그룹번호'].apply(lambda x:int(str(x)[3:]))

In [6352]:
_train.columns

Index(['기준년월', '그룹번호', '종목번호', '그룹내고객수', '매수비율_x', '시장구분_mean', '종목시가_mean',
       '종목고가_mean', '종목저가_mean', '종목종가_mean', '거래량_mean', '거래금액_만원단위_mean',
       '산업코드_mean', '종목시가_std', '종목고가_std', '종목저가_std', '종목종가_std', '거래량_std',
       '거래금액_만원단위_std', '종목시가_cv', '종목고가_cv', '종목저가_cv', '종목종가_cv', '거래량_cv',
       '거래금액_만원단위_cv', '종목시가_monthly_rtn', '종목고가_monthly_rtn',
       '종목저가_monthly_rtn', '종목종가_monthly_rtn', '거래량_monthly_rtn',
       '거래금액_만원단위_monthly_rtn', 'rtn_mean', 'rtn_max', 'rtn_std', '매수비율_y',
       '검색변화율', '순매수금액(만)'],
      dtype='object')

In [6353]:
_train.drop(['종목시가_mean','종목고가_mean','종목저가_mean','종목종가_mean','종목시가_std','종목고가_std','종목저가_std','거래량_std','거래금액_만원단위_std', '종목시가_cv', '종목고가_cv', '종목저가_cv','종목시가_monthly_rtn', '종목고가_monthly_rtn','종목저가_monthly_rtn'],axis=1,inplace=True)

In [6354]:
_train

,기준년월,그룹번호,종목번호,그룹내고객수,매수비율_x,시장구분_mean,거래량_mean,거래금액_만원단위_mean,산업코드_mean,종목종가_std,종목종가_cv,거래량_cv,거래금액_만원단위_cv,종목종가_monthly_rtn,거래량_monthly_rtn,거래금액_만원단위_monthly_rtn,rtn_mean,rtn_max,rtn_std,매수비율_y,검색변화율,순매수금액(만)
0,201907,1,0,288,0.024306,1.0,2.993388e+06,2.244919e+07,260.0,3852.005278,0.051678,0.309477,0.331491,1.098571,1.049194,1.148590,0.004346,0.045321,0.022784,0.000000,0.000000,0.00
1,201907,1,1,288,0.010417,1.0,3.604345e+05,1.821396e+06,260.0,3676.161052,0.075398,1.303390,1.368340,0.843870,1.987147,1.643043,-0.005577,0.131715,0.043531,0.000000,0.000000,151.74
2,201907,1,2,288,0.010417,1.0,2.563905e+05,1.031642e+06,241.0,1045.279243,0.026003,0.250455,0.259252,0.907602,0.746717,0.678284,-0.003587,0.019083,0.013425,0.000000,0.000000,0.00
3,201907,1,3,288,0.013889,1.0,5.077075e+05,6.836587e+06,301.0,4241.592474,0.031460,0.460072,0.456656,0.897163,1.486736,1.347463,-0.004754,0.025830,0.014860,0.000000,0.000000,484.97
4,201907,1,4,288,0.045139,1.0,7.990078e+06,3.696300e+07,264.0,735.686875,0.015919,0.263625,0.264443,0.973176,1.130838,1.105007,-0.001004,0.017563,0.011626,0.000000,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35110,202007,48,40,163,0.000000,1.0,1.999277e+05,1.507454e+07,211.0,17534.552356,0.023325,0.320287,0.330174,1.011968,0.744472,0.744076,-0.001016,0.048877,0.024134,0.018405,-0.265752,2827.23
35111,202007,48,692,163,0.000000,1.0,4.495869e+05,4.156590e+05,260.0,265.119635,0.029115,0.708958,0.748988,1.100679,1.974541,2.167452,0.004848,0.042324,0.014804,0.018405,-0.366284,0.00
35112,202007,48,370,163,0.000000,1.0,1.800092e+06,3.880514e+07,202.0,59359.076812,0.334417,0.974620,1.351223,2.158672,0.404047,0.895064,0.044161,0.350000,0.119078,0.036810,1.981800,940.66
35113,202007,48,416,163,0.000000,1.0,1.533328e+06,1.347315e+06,649.0,162.716253,0.018473,0.366472,0.356423,0.972004,2.490301,2.426683,0.000395,0.018326,0.015063,0.030675,-0.292038,470.65


In [6355]:
train = _train[_train['기준년월'] != "202007"]

In [6356]:
test = _train[_train['기준년월'] == "202007"]

In [6357]:
train_y = train['매수비율_x']

In [6358]:
train_x = train.drop("매수비율_x",axis=1)

In [6359]:
test_x = test.drop("매수비율_x",axis=1)

In [6360]:
train_x.drop(['그룹내고객수'],axis=1,inplace=True)
test_x.drop(['그룹내고객수'],axis=1,inplace=True)

In [6361]:
from lightgbm import LGBMRegressor

In [6362]:
model = LGBMRegressor(num_leaves=2,num_iterations=10000)

In [6363]:
model.fit(train_x.values,train_y.values)

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:151: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMRegressor(num_iterations=10000, num_leaves=2)

In [6364]:
pred = model.predict(test_x.values)

In [6365]:
pred

array([0.2538057 , 0.01708947, 0.01226445, ..., 0.06187559, 0.03032838,
       0.03570605])

In [6366]:
sorted([[x,test_x.columns[x],model.feature_importances_[x]] for x in range(len(model.feature_importances_))],key=lambda x:x[2],reverse=True)

[[1, '그룹번호', 6054],
 [19, '순매수금액(만)', 1220],
 [5, '거래금액_만원단위_mean', 647],
 [17, '매수비율_y', 475],
 [2, '종목번호', 298],
 [6, '산업코드_mean', 145],
 [9, '거래량_cv', 128],
 [16, 'rtn_std', 114],
 [10, '거래금액_만원단위_cv', 103],
 [11, '종목종가_monthly_rtn', 102],
 [14, 'rtn_mean', 99],
 [0, '기준년월', 95],
 [7, '종목종가_std', 89],
 [18, '검색변화율', 89],
 [4, '거래량_mean', 79],
 [15, 'rtn_max', 79],
 [8, '종목종가_cv', 69],
 [12, '거래량_monthly_rtn', 69],
 [13, '거래금액_만원단위_monthly_rtn', 31],
 [3, '시장구분_mean', 15]]

In [6367]:
test_x['매수비율_x'] = pred

test_x['매수비율_x'] = test_x['매수비율_x']* test_x['매수비율_y'] * (test_x['순매수금액(백만)']/100 + test_x['거래금액_만원단위_mean'])/ 2

In [6368]:
test_x

,기준년월,그룹번호,종목번호,시장구분_mean,거래량_mean,거래금액_만원단위_mean,산업코드_mean,종목종가_std,종목종가_cv,거래량_cv,거래금액_만원단위_cv,종목종가_monthly_rtn,거래량_monthly_rtn,거래금액_만원단위_monthly_rtn,rtn_mean,rtn_max,rtn_std,매수비율_y,검색변화율,순매수금액(만),매수비율_x
30200,202007,1,0,1.0,2902662.00,24298966.85,260.0,1080.496962,0.012929,0.284061,0.284355,0.974209,1.904983,1.844654,-0.001263,0.015116,0.013533,0.215278,0.230990,8906.26,0.253806
30201,202007,1,222,1.0,1118361.85,3472320.80,260.0,1205.085168,0.039670,0.981466,1.045712,1.168421,8.514745,9.774819,0.003812,0.070050,0.031143,0.013889,0.043642,0.00,0.017089
30202,202007,1,1,1.0,230033.35,1295989.95,260.0,1649.082680,0.029902,1.152757,1.193369,1.129032,14.392000,15.719993,0.002016,0.043560,0.020415,0.013889,-0.041340,0.00,0.012264
30203,202007,1,860,1.0,9434310.30,3783387.85,301.0,260.447489,0.068314,1.475360,1.549541,1.156773,0.145114,0.164409,0.004807,0.254213,0.072080,0.010417,-0.332703,0.00,0.010648
30204,202007,1,3,1.0,2202682.70,25180255.95,301.0,10685.182805,0.097622,0.664520,0.713320,1.252546,2.755767,3.434068,0.011880,0.091323,0.029633,0.031250,0.383200,0.00,0.037693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35110,202007,48,40,1.0,199927.70,15074543.50,211.0,17534.552356,0.023325,0.320287,0.330174,1.011968,0.744472,0.744076,-0.001016,0.048877,0.024134,0.018405,-0.265752,2827.23,0.038888
35111,202007,48,692,1.0,449586.90,415659.00,260.0,265.119635,0.029115,0.708958,0.748988,1.100679,1.974541,2.167452,0.004848,0.042324,0.014804,0.018405,-0.366284,0.00,0.015839
35112,202007,48,370,1.0,1800091.65,38805135.50,202.0,59359.076812,0.334417,0.974620,1.351223,2.158672,0.404047,0.895064,0.044161,0.350000,0.119078,0.036810,1.981800,940.66,0.061876
35113,202007,48,416,1.0,1533328.00,1347315.40,649.0,162.716253,0.018473,0.366472,0.356423,0.972004,2.490301,2.426683,0.000395,0.018326,0.015063,0.030675,-0.292038,470.65,0.030328


In [6369]:
test_x['그룹번호'] = test_x['그룹번호'].apply(lambda x:("0"+str(x))[-2:])

In [6370]:
test_x['그룹번호'] = test_x['그룹번호'].apply(lambda x:"MAD"+str(x))

In [6371]:
stock_name_list_reverse

,0
0,A000660
1,A001820
2,A004020
3,A005380
4,A005930
...,...
1061,A122350
1062,A024900
1063,A064240
1064,A086890


In [6372]:
test_x['종목번호'] = test_x['종목번호'].apply(lambda x:stock_name_list_reverse.loc[x,0])

In [6373]:
test_x

,기준년월,그룹번호,종목번호,시장구분_mean,거래량_mean,거래금액_만원단위_mean,산업코드_mean,종목종가_std,종목종가_cv,거래량_cv,거래금액_만원단위_cv,종목종가_monthly_rtn,거래량_monthly_rtn,거래금액_만원단위_monthly_rtn,rtn_mean,rtn_max,rtn_std,매수비율_y,검색변화율,순매수금액(만),매수비율_x
30200,202007,MAD01,A000660,1.0,2902662.00,24298966.85,260.0,1080.496962,0.012929,0.284061,0.284355,0.974209,1.904983,1.844654,-0.001263,0.015116,0.013533,0.215278,0.230990,8906.26,0.253806
30201,202007,MAD01,A000990,1.0,1118361.85,3472320.80,260.0,1205.085168,0.039670,0.981466,1.045712,1.168421,8.514745,9.774819,0.003812,0.070050,0.031143,0.013889,0.043642,0.00,0.017089
30202,202007,MAD01,A001820,1.0,230033.35,1295989.95,260.0,1649.082680,0.029902,1.152757,1.193369,1.129032,14.392000,15.719993,0.002016,0.043560,0.020415,0.013889,-0.041340,0.00,0.012264
30203,202007,MAD01,A003620,1.0,9434310.30,3783387.85,301.0,260.447489,0.068314,1.475360,1.549541,1.156773,0.145114,0.164409,0.004807,0.254213,0.072080,0.010417,-0.332703,0.00,0.010648
30204,202007,MAD01,A005380,1.0,2202682.70,25180255.95,301.0,10685.182805,0.097622,0.664520,0.713320,1.252546,2.755767,3.434068,0.011880,0.091323,0.029633,0.031250,0.383200,0.00,0.037693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35110,202007,MAD48,A207940,1.0,199927.70,15074543.50,211.0,17534.552356,0.023325,0.320287,0.330174,1.011968,0.744472,0.744076,-0.001016,0.048877,0.024134,0.018405,-0.265752,2827.23,0.038888
35111,202007,MAD48,A272210,1.0,449586.90,415659.00,260.0,265.119635,0.029115,0.708958,0.748988,1.100679,1.974541,2.167452,0.004848,0.042324,0.014804,0.018405,-0.366284,0.00,0.015839
35112,202007,MAD48,A285130,1.0,1800091.65,38805135.50,202.0,59359.076812,0.334417,0.974620,1.351223,2.158672,0.404047,0.895064,0.044161,0.350000,0.119078,0.036810,1.981800,940.66,0.061876
35113,202007,MAD48,A316140,1.0,1533328.00,1347315.40,649.0,162.716253,0.018473,0.366472,0.356423,0.972004,2.490301,2.426683,0.000395,0.018326,0.015063,0.030675,-0.292038,470.65,0.030328


In [6374]:
_tmp = test_x[['그룹번호','종목번호','매수비율_x']]

In [6375]:
_tmp = _tmp.pivot_table(index=['그룹번호','종목번호'])

In [6376]:
final_list = stock[['종목번호','20년7월TOP3대상여부']]
final_list = final_list[final_list['20년7월TOP3대상여부'] == "Y"]
final_list=list(final_list['종목번호'].unique())

In [6377]:
result = {}
for i in _tmp.index.levels[0]:
    result[i] = []

In [6378]:
for i in _tmp.index.levels[0]:
    __tmp = _tmp.loc[(i)]
    __tmp = __tmp.sort_values("매수비율_x",ascending=False)
    cnt=0
    for j in __tmp.index:
        if cnt == 3:
            break
        if j in final_list:
            result[i].append(j)
            cnt+=1

In [6379]:
for i in result.keys():
    result[i].sort()

In [6380]:
final = pd.merge(pd.DataFrame(final_list,columns=['종목번호']),stock[['종목번호','종목명']].drop_duplicates(),how='left')

In [6381]:
result = pd.DataFrame(result).T
result.columns = ['종목번호1','종목번호2','종목번호3']
result.index.name="그룹번호"

In [6382]:
result

,종목번호1,종목번호2,종목번호3
그룹번호,,,
MAD01,A000660,A005930,A005935
MAD02,A003490,A005930,A105560
MAD03,A003000,A007570,A019170
MAD04,A000660,A003000,A005930
MAD05,A035420,A035720,A068270
MAD06,A005930,A035420,A035720
MAD07,A005930,A010950,A096770
MAD08,A003000,A007570,A019170
MAD09,A035720,A051910,A068270


In [6383]:
result.to_csv("result.csv")